In [10]:
import os
import subprocess
import plotly.graph_objects as go

from collections import defaultdict

In [2]:
class Instance:
    def __init__(self, name: str, tool_count: int, slot_count: int, matrix: list[list[int]], soa_cost: int = None, solution: list[int] = None):
        self.name = name
        self.tool_count = tool_count
        self.slot_count = slot_count
        self.matrix = matrix
        self.soa_cost= soa_cost
        self.solution = solution
        
    def mount_lkh_par_prompt(self, dir: str = "") -> str:
        return (
            f"PROBLEM_FILE = {dir}{self.name}.tsp\n"
            f"OUTPUT_TOUR_FILE = {dir}{self.name}.tour\n"
            "RUNS = 1\n"
        )

    def mount_lkh_tsp_prompt(self) -> str:
        header = (
            f"NAME: {self.name}\n"
            "TYPE: TSP\n"
            f"DIMENSION: {self.tool_count}\n"
            "EDGE_WEIGHT_TYPE: EXPLICIT\n"
            "EDGE_WEIGHT_FORMAT: FULL_MATRIX\n"
            "EDGE_WEIGHT_SECTION\n"
        )
        matrix = "\n".join(" ".join(str(-x) for x in row) for row in self.matrix)
        return f"{header}{matrix}\nEOF"
        
    def calculate_cost(self) -> int:
        tool_index = {tool: idx for idx, tool in enumerate(self.solution)}
        total_cost = 0

        for i in range(self.tool_count):
            for j in range(i + 1, self.tool_count):
                freq = self.matrix[i][j]
                if freq == 0:
                    continue

                dist = abs(tool_index[i] - tool_index[j])
                circular_dist = min(dist, (self.tool_count - dist) + (self.slot_count - self.tool_count))
                total_cost += freq * circular_dist

        return total_cost

In [ ]:
def plot_cost_comparison(plot_data: dict, template="plotly_white", prefix: str = None):
    import re
    def extract_number(name):
        numbers = re.findall(r'\d+', name)
        return int(numbers[0]) if numbers else float('inf')

    # Filter and sort only instances matching the prefix (if provided)
    filtered_instances = [
        inst for inst in plot_data.keys()
        if prefix is None or inst.startswith(prefix)
    ]
    sorted_instances = sorted(filtered_instances, key=extract_number)

    # Extract costs using the sorted + filtered instances
    lkh_costs = [plot_data[inst]["LKH"] for inst in sorted_instances]
    soa_costs = [plot_data[inst]["SOA"] for inst in sorted_instances]

    # Plot
    fig = go.Figure(data=[
        go.Bar(
            y=sorted_instances,
            x=soa_costs,
            name='SOA',
            orientation='h',
            marker_color='#FF0000',
            text=[f"{x:.2f}" for x in soa_costs],
            textposition='outside',
            hoverinfo='x+y+name'
        ),
        go.Bar(
            y=sorted_instances,
            x=lkh_costs,
            name='LKH',
            orientation='h',
            marker_color='#0000FF',
            text=[f"{x:.2f}" for x in lkh_costs],
            textposition='outside',
            hoverinfo='x+y+name'
        )
    ])

    fig.update_layout(
        title='Cost Comparison (LKH vs SOA)',
        xaxis_title='Cost',
        yaxis_title='Instance',
        barmode='group',
        template=template,
        width=1000,
        height=600 + len(sorted_instances) * 10,
        font=dict(size=12),
        margin=dict(l=100, r=50, t=50, b=50)
    )

    fig.show()


In [3]:
solutions = []
instances = {}
main_dir = "/home/pedro/Pessoal/MSc/tip/instances"
for root, dirs, files in os.walk(main_dir):
    if root != main_dir:
        txt_files = [f for f in files if f.endswith(".txt")]
        for file in txt_files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                tool_count, slot_count = map(int, lines[0].strip().split())
                instance = Instance(
                    name=file[:-4],
                    tool_count=tool_count,
                    slot_count=slot_count,
                    soa_cost=int(lines[2].replace('\n', '').strip()),
                    matrix=[[int(x) for x in line.strip().split()] for line in lines[3:] if line != '\n']
                )
                instances[instance.name] = instance

In [4]:
lkh_path = "/home/pedro/Pessoal/MSc/tip/src/LKH/LKH"
for name, instance in instances.items():
    dir = "/home/pedro/Pessoal/MSc/tip/src/LKH/IO/"
    par_content = instance.mount_lkh_par_prompt(dir)
    tsp_content = instance.mount_lkh_tsp_prompt()

    par_path = os.path.join(dir, f"{name}.par")
    with open(par_path, "w", encoding="utf-8") as f:
        f.write(par_content)

    tsp_path = os.path.join(dir, f"{name}.tsp")
    with open(tsp_path, "w", encoding="utf-8") as f:
        f.write(tsp_content)
        
    print(f"Running LKH for instance {name}...")
    subprocess.run([lkh_path, par_path], check=True)

Running LKH for instance o-10_t_2...
PARAMETER_FILE = /home/pedro/Pessoal/MSc/tip/src/LKH/IO/o-10_t_2.par
Reading PROBLEM_FILE: "/home/pedro/Pessoal/MSc/tip/src/LKH/IO/o-10_t_2.tsp" ... done
ASCENT_CANDIDATES = 9
BACKBONE_TRIALS = 0
BACKTRACKING = NO
# CANDIDATE_FILE =
CANDIDATE_SET_TYPE = ALPHA
# EDGE_FILE =
EXCESS = 0.1
EXTRA_CANDIDATES = 0 
EXTRA_CANDIDATE_SET_TYPE = QUADRANT
GAIN23 = YES
GAIN_CRITERION = YES
INITIAL_PERIOD = 100
INITIAL_STEP_SIZE = 1
INITIAL_TOUR_ALGORITHM = WALK
# INITIAL_TOUR_FILE = 
INITIAL_TOUR_FRACTION = 1.000
# INPUT_TOUR_FILE = 
KICK_TYPE = 0
KICKS = 1
# MAX_BREADTH =
MAX_CANDIDATES = 5 
MAX_SWAPS = 10
MAX_TRIALS = 10
# MERGE_TOUR_FILE =
MOVE_TYPE = 5
# NONSEQUENTIAL_MOVE_TYPE = 5
# OPTIMUM =
OUTPUT_TOUR_FILE = /home/pedro/Pessoal/MSc/tip/src/LKH/IO/o-10_t_2.tour
PATCHING_A = 1 
PATCHING_C = 0 
# PI_FILE = 
POPMUSIC_INITIAL_TOUR = NO
POPMUSIC_MAX_NEIGHBORS = 5
POPMUSIC_SAMPLE_SIZE = 10
POPMUSIC_SOLUTIONS = 50
POPMUSIC_TRIALS = 1
# POPULATION_SIZE = 0
PRECISI

In [5]:
lkh_path = "/home/pedro/Pessoal/MSc/tip/src/LKH/LKH"
plot_data = {}
for name, instance in instances.items():
    result_path = os.path.join("/home/pedro/Pessoal/MSc/tip/src/LKH/IO/", f"{name}.tour")
    with open(result_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        solution = [int(line.strip()) - 1 for line in lines[6:-2] if line != '\n']
        instance.solution = solution
        plot_data[name] = {
            "LKH": instance.calculate_cost(),
            "SOA": instance.soa_cost
        }

In [ ]:
plot_cost_comparison(plot_data, prefix="anjos")